In [44]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from fake_useragent import UserAgent
import pandas as pd
import math
from random import choice
from selenium.webdriver.common.proxy import *

def get_proxy():
    html = requests.get('https://free-proxy-list.net/',headers={'User-Agent': UserAgent().firefox}).text
    soup = BeautifulSoup(html, 'lxml')

    trs = soup.find('table', id='proxylisttable').find_all('tr')[1:11]

    proxies = []

    for tr in trs:
        tds = tr.find_all('td')
        if 'yes' in tds[6].text.strip():
            ip = tds[0].text.strip()
            port = tds[1].text.strip()
            schema = 'https' if 'yes' in tds[6].text.strip() else 'http'
            proxy = {'schema': schema, 'address': ip + ':' + port}
            proxies.append(proxy)

    return choice(proxies)

def get_html(url, browser):
    browser.get(url)
    requiredHtml = browser.page_source
    #r = requests.get(url,headers={'User-Agent': UserAgent().chrome})
    return requiredHtml

def numfromstr(s):
    l = len(s)
    i = 0
    s_int = ''
    while i < l:
        if '0' <= s[i] <= '9':
            s_int += s[i]
        i = i + 1
    if s_int:
        return float(s_int)
    else:
        return 0
    
def get_list(url, browser):
    soup = BeautifulSoup(get_html(url, browser), 'lxml')
    num_items = int(soup.find('span', class_="total many").span.text)
    num_pages = min(50,math.ceil(num_items/100))
    print(num_pages)
    divs = soup.find_all('div', class_="dtList i-dtList j-card-item")
    links = []
    for d in divs:
        link = d.find('a', class_="ref_goods_n_p")['href'].strip()
        links.append(link)


    if num_pages > 1:
        for page in range(2,num_pages+1):
            soup = BeautifulSoup(get_html(url + '&page=' + str(page), browser), 'lxml')
            divs = soup.find_all('div', class_="dtList i-dtList j-card-item")
            for d in divs:
                link = d.find('a', class_="ref_goods_n_p")['href'].strip()
                links.append(link)
        
    return links   

def get_item(html, browser):
    soup = BeautifulSoup(html, 'lxml')
    article=soup.find('span', class_='j-article').text.strip()
    name=soup.find('span', class_='name').text.strip()
    brand=soup.find('span', class_='brand').text.strip()
    price = numfromstr(soup.find('span', class_='final-cost').text.strip())
    orders_count = numfromstr(soup.find('span', class_='j-orders-count').text.strip())
    comments = int(soup.find('a', id="comments_reviews_link").i.text.strip())
    rating = int(soup.find('div', class_="product-rating").p.text.strip())
    #first_comment = soup.find('div', class_="time text-base-gray").p.text.strip()
    browser.find_element_by_link_text('дате').click()
    first_comment = browser.find_element_by_class_name('time.text-base-gray').get_attribute('content')
    
    item = {'article': article, 'brand': brand,'name': name,  'price': price,  'comments': comments,  'orders_count': orders_count,  'rating': rating,  'first_comment': first_comment}
    return item

def main():
    url = 'https://www.wildberries.ru/catalog/0/search.aspx?kind=2&subject=4&search=%D0%BA%D1%83%D0%BF%D0%B0%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%20%D0%B6%D0%B5%D0%BD%D1%81%D0%BA%D0%B8%D0%B9&sort=popular'

    # путь к драйверу chrome
    firefoxdriver = 'C:\Work\drive\geckodriver.exe'
    options = webdriver.FirefoxOptions()
    options.headless = True
    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", UserAgent().random)
    
    PROXY = get_proxy()['address']
    print(PROXY)
    
    proxy = Proxy({
        'proxyType': ProxyType.MANUAL,
        'httpProxy': PROXY,
        'ftpProxy': PROXY,
        'sslProxy': PROXY,
        'noProxy':''})
    
    
    browser = webdriver.Firefox(executable_path=firefoxdriver, options=options, proxy=proxy, firefox_profile=profile) 
    
    # you have to use remote, otherwise you'll have to code it yourself in python to 
    #browser = webdriver.Remote("http://localhost:4444/wd/hub", webdriver.DesiredCapabilities.FIREFOX)


    
    links = pd.read_csv('links.csv',header=None)[0].tolist()
    links = links[:10]
    
    
    i=0
    items = []
    for link in links:
        #print(link)
        item = get_item(get_html(link, browser), browser)
        #print(item)
        items.append(item)
        i+=1
        if i == int(i/100):
            df = pd.DataFrame(items)
            df['link']=links
            df.to_excel('parsing.xlsx')
            print(i)
            browser.close()
            PROXY = get_proxy()['address']
            proxy = Proxy({
                'proxyType': ProxyType.MANUAL,
                'httpProxy': PROXY,
                'ftpProxy': PROXY,
                'sslProxy': PROXY,
                'noProxy':''})
            browser = webdriver.Firefox(executable_path=firefoxdriver, options=options, proxy=proxy, firefox_profile=profile)
            
    df = pd.DataFrame(items)
    df['link']=links
    df.to_excel('parsing.xlsx')
    
    #print(len(links))
    #print(links[:10])
    
    browser.close()
    
    
def save_list():
    url = 'https://www.wildberries.ru/catalog/0/search.aspx?kind=2&subject=4&search=%D0%BA%D1%83%D0%BF%D0%B0%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%20%D0%B6%D0%B5%D0%BD%D1%81%D0%BA%D0%B8%D0%B9&sort=popular'

    # путь к драйверу chrome
    firefoxdriver = 'C:\Work\drive\geckodriver.exe'
    options = webdriver.FirefoxOptions()
    options.headless = True
    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", UserAgent().random)

    
    PROXY = get_proxy()['address']
    print(PROXY)
    
    proxy = Proxy({
        'proxyType': ProxyType.MANUAL,
        'httpProxy': PROXY,
        'ftpProxy': PROXY,
        'sslProxy': PROXY,
        'noProxy':''})
    
    browser = webdriver.Firefox(executable_path=firefoxdriver, options=options, proxy=proxy, firefox_profile=profile) 
    
    
    links = get_list(url, browser)
    
    df = pd.DataFrame(links).to_csv('links2.csv', index=False, header=False)

    browser.close()     




In [47]:
def get_proxy():
    html = requests.get('https://free-proxy-list.net/',headers={'User-Agent': UserAgent().firefox}).text
    soup = BeautifulSoup(html, 'lxml')

    trs = soup.find('table', id='proxylisttable').find_all('tr')[1:11]

    proxies = []

    for tr in trs:
        tds = tr.find_all('td')
        if 'yes' in tds[6].text.strip():
            ip = tds[0].text.strip()
            port = tds[1].text.strip()
            schema = 'https' if 'yes' in tds[6].text.strip() else 'http'
            proxy = {'schema': schema, 'address': ip + ':' + port}
            proxies.append(proxy)

    return choice(proxies)

In [53]:
def get_html(url, browser):
    browser.get(url)
    requiredHtml = browser.page_source
    #r = requests.get(url,headers={'User-Agent': UserAgent().chrome})
    return requiredHtml

In [4]:
def numfromstr(s):
    l = len(s)
    i = 0
    s_int = ''
    while i < l:
        if '0' <= s[i] <= '9':
            s_int += s[i]
        i = i + 1
    if s_int:
        return float(s_int)
    else:
        return 0


In [5]:
def get_list(url, browser):
    soup = BeautifulSoup(get_html(url, browser), 'lxml')
    num_items = int(soup.find('span', class_="total many").span.text)
    num_pages = min(50,math.ceil(num_items/100))
    print(num_pages)
    divs = soup.find_all('div', class_="dtList i-dtList j-card-item")
    links = []
    for d in divs:
        link = d.find('a', class_="ref_goods_n_p")['href'].strip()
        links.append(link)


    if num_pages > 1:
        for page in range(2,num_pages+1):
            soup = BeautifulSoup(get_html(url + '&page=' + str(page), browser), 'lxml')
            divs = soup.find_all('div', class_="dtList i-dtList j-card-item")
            for d in divs:
                link = d.find('a', class_="ref_goods_n_p")['href'].strip()
                links.append(link)
        
    return links     

In [39]:
def get_item(html, browser):
    soup = BeautifulSoup(html, 'lxml')
    article=soup.find('span', class_='j-article').text.strip()
    name=soup.find('span', class_='name').text.strip()
    brand=soup.find('span', class_='brand').text.strip()
    price = numfromstr(soup.find('span', class_='final-cost').text.strip())
    orders_count = numfromstr(soup.find('span', class_='j-orders-count').text.strip())
    comments = int(soup.find('a', id="comments_reviews_link").i.text.strip())
    rating = int(soup.find('div', class_="product-rating").p.text.strip())
    #first_comment = soup.find('div', class_="time text-base-gray").p.text.strip()
    browser.find_element_by_link_text('дате').click()
    first_comment = browser.find_element_by_class_name('time.text-base-gray').get_attribute('content')
    
    item = {'article': article, 'brand': brand,'name': name,  'price': price,  'comments': comments,  'orders_count': orders_count,  'rating': rating,  'first_comment': first_comment}
    return item



In [73]:
def main():
    url = 'https://www.wildberries.ru/catalog/0/search.aspx?kind=2&subject=4&search=%D0%BA%D1%83%D0%BF%D0%B0%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%20%D0%B6%D0%B5%D0%BD%D1%81%D0%BA%D0%B8%D0%B9&sort=popular'

    # путь к драйверу chrome
    firefoxdriver = 'C:\Work\drive\geckodriver.exe'
    options = webdriver.FirefoxOptions()
    options.headless = True
    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", UserAgent().random)
    
    PROXY = get_proxy()['address']
    print(PROXY)
    
    proxy = Proxy({
        'proxyType': ProxyType.MANUAL,
        'httpProxy': PROXY,
        'ftpProxy': PROXY,
        'sslProxy': PROXY,
        'noProxy':''})
    
    
    browser = webdriver.Firefox(executable_path=firefoxdriver, options=options, proxy=proxy, firefox_profile=profile) 
    
    # you have to use remote, otherwise you'll have to code it yourself in python to 
    #browser = webdriver.Remote("http://localhost:4444/wd/hub", webdriver.DesiredCapabilities.FIREFOX)


    
    links = pd.read_csv('links.csv',header=None)[0].tolist()
    links = links[:10]
    
    
    i=0
    items = []
    for link in links:
        #print(link)
        item = get_item(get_html(link, browser), browser)
        #print(item)
        items.append(item)
        i+=1
        if i == int(i/100):
            df = pd.DataFrame(items)
            df['link']=links
            df.to_excel('parsing.xlsx')
            print(i)
            browser.close()
            PROXY = get_proxy()['address']
            proxy = Proxy({
                'proxyType': ProxyType.MANUAL,
                'httpProxy': PROXY,
                'ftpProxy': PROXY,
                'sslProxy': PROXY,
                'noProxy':''})
            browser = webdriver.Firefox(executable_path=firefoxdriver, options=options, proxy=proxy, firefox_profile=profile)
            
    df = pd.DataFrame(items)
    df['link']=links
    df.to_excel('parsing.xlsx')
    
    #print(len(links))
    #print(links[:10])
    
    browser.close()
    

In [74]:
main()

87.99.93.100:43944


In [63]:
def save_list():
    url = 'https://www.wildberries.ru/catalog/0/search.aspx?kind=2&subject=4&search=%D0%BA%D1%83%D0%BF%D0%B0%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%20%D0%B6%D0%B5%D0%BD%D1%81%D0%BA%D0%B8%D0%B9&sort=popular'

    # путь к драйверу chrome
    firefoxdriver = 'C:\Work\drive\geckodriver.exe'
    options = webdriver.FirefoxOptions()
    options.headless = True
    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", UserAgent().random)

    
    PROXY = get_proxy()['address']
    print(PROXY)
    
    proxy = Proxy({
        'proxyType': ProxyType.MANUAL,
        'httpProxy': PROXY,
        'ftpProxy': PROXY,
        'sslProxy': PROXY,
        'noProxy':''})
    
    browser = webdriver.Firefox(executable_path=firefoxdriver, options=options, proxy=proxy, firefox_profile=profile) 
    
    
    links = get_list(url, browser)
    
    df = pd.DataFrame(links).to_csv('links2.csv', index=False, header=False)

    browser.close()   

In [64]:
save_list()

41.164.247.186:53281
50


In [37]:
url = 'https://www.wildberries.ru/catalog/5275302/detail.aspx?targetUrl=SG'

    # путь к драйверу chrome
firefoxdriver = 'C:\Work\drive\geckodriver.exe'
options = webdriver.FirefoxOptions()
options.add_argument('headless')  # для открытия headless-браузера
options.add_argument("window-size=800,400")
options.add_argument(f'user-agent={UserAgent().firefox}')

PROXY = get_proxy()['address']
    
proxy = Proxy({
    'proxyType': ProxyType.MANUAL,
    'httpProxy': PROXY,
    'ftpProxy': PROXY,
    'sslProxy': PROXY,
    'noProxy':''})
    
    
browser = webdriver.Firefox(executable_path=firefoxdriver, options=options, proxy=proxy) 


soup = BeautifulSoup(get_html(url, browser), 'html5')
browser.find_element_by_link_text('дате').click()
s = browser.find_element_by_class_name('time.text-base-gray').get_attribute('content')
s

'2017-08-28T18:02:20.0960000'

In [22]:
s

In [31]:
s = browser.find_element_by_class_name('time text-base-gray')

NoSuchElementException: Message: Unable to locate element: .time text-base-gray


In [38]:
soup.find('div', class_='time.text-base-gray')

In [33]:
soup.find('script', id="productQuestionTmpl")

In [36]:
script
re.findall(r'[\'\"]company[\'\"]\s*\:\s*[\'\"]([^\'\"]*)[\'\"]',
                       script.text, flags=re.I)

<script id="productQuestionTmpl" type="text/x-jsrender">
        <div class="question">
            <div class="question-profile">
                <div class="avatar-block">
                    <div class="avatar-layout">
                        <img class="avatar" src="{{:~wbSettings.staticSiteUrl}}/i/blank.gif" data-original='{{:~PersonalPhoto(wbUserId, wbUserDetails.hasPhoto)}}' alt="avatar">
                    </div>
                    <div class="country country-{{toLower:wbUserDetails.country}}">{{:wbUserDetails.country}}</div>
                </div>
            </div>
            <div class="question-content">
                <div class="question-content-user-and-time">
                    <div itemprop="author" class="author">
                        <label data-user-id="{{:wbUserId}}" class="text-base-bold">{{:wbUserDetails.name}}</label>
                    </div>
                    <div class="time text-base-gray">
                        <meta content="{{:createdDate}}">